In [25]:
import pandas as pd
import glob # used to read file directories, there are alternatives: pathlib.Path.rglob or os.walk

In [26]:
def load_xls_from_dir(path):
    files= glob.glob(path, recursive=True)
    df= pd.DataFrame()
    for f in files:
        #Option 1: read just the first sheet of book
        #data=pd.read_excel(f) 
        #df=df.append(data, sort=False)
        #Option 2: loop through each tab in each workbook and add id fields
        xl = pd.ExcelFile(f)
        for s in xl.sheet_names:
            data=xl.parse(s)
            data["ori_file"]= f
            data["ori_sheet"]=s
            df=df.append(data, sort=False)
    return df
   

In [27]:
df= load_xls_from_dir("./testdata/batch/*.xlsx")

In [28]:
df

,Customer_ID,Date,Transaction_ID,Result,ori_file,ori_sheet
0,474480,2019-04-01,1,Authorised,./testdata/batch/Q2.xlsx,Apr
1,372053,2019-04-01,2,Authorised,./testdata/batch/Q2.xlsx,Apr
2,317920,2019-04-01,3,Authorised,./testdata/batch/Q2.xlsx,Apr
3,172826,2019-04-02,4,Authorised,./testdata/batch/Q2.xlsx,Apr
4,410735,2019-04-02,5,Authorised,./testdata/batch/Q2.xlsx,Apr
...,...,...,...,...,...,...
1928,289307,2019-03-31,5761,Authorised,./testdata/batch/Q1.xlsx,Mar
1929,301587,2019-03-31,5762,Authorised,./testdata/batch/Q1.xlsx,Mar
1930,304898,2019-03-31,5763,Authorised,./testdata/batch/Q1.xlsx,Mar
1931,463912,2019-03-31,5764,Authorised,./testdata/batch/Q1.xlsx,Mar


In [35]:
df.columns=[x.lower() for x in df.columns.to_list()]
df.columns=df.columns.str.replace(' ','_')
df.head()

,customer_id,date,transaction_id,result,ori_file,ori_sheet
683,328647,2019-06-30,3954,Authorised,./testdata/batch/Q2.xlsx,Jun
657,153100,2019-06-30,3928,Authorised,./testdata/batch/Q2.xlsx,Jun
655,306857,2019-06-30,3926,Authorised,./testdata/batch/Q2.xlsx,Jun
654,94990,2019-06-30,3925,Authorised,./testdata/batch/Q2.xlsx,Jun
653,57777,2019-06-30,3924,Authorised,./testdata/batch/Q2.xlsx,Jun


In [36]:
df.sort_values('date', axis=0, ascending=False, inplace=True)
df[0:10]

,customer_id,date,transaction_id,result,ori_file,ori_sheet
683,328647,2019-06-30,3954,Authorised,./testdata/batch/Q2.xlsx,Jun
637,260978,2019-06-30,3908,Authorised,./testdata/batch/Q2.xlsx,Jun
672,57941,2019-06-30,3943,Authorised,./testdata/batch/Q2.xlsx,Jun
659,34300,2019-06-30,3930,Authorised,./testdata/batch/Q2.xlsx,Jun
682,408881,2019-06-30,3953,Authorised,./testdata/batch/Q2.xlsx,Jun
681,98175,2019-06-30,3952,Authorised,./testdata/batch/Q2.xlsx,Jun
680,369736,2019-06-30,3951,Authorised,./testdata/batch/Q2.xlsx,Jun
679,97450,2019-06-30,3950,Rejected,./testdata/batch/Q2.xlsx,Jun
678,79149,2019-06-30,3949,Authorised,./testdata/batch/Q2.xlsx,Jun
677,212854,2019-06-30,3948,Authorised,./testdata/batch/Q2.xlsx,Jun


This is not the best way to test for duplicates, but can do the trick

In [39]:
df2= df.groupby(["transaction_id"], as_index = False ).count()
# If you dont include the as_index, the DF will have nested indexes, so to get the data use:
# list= df2.index.get_level_values(1).tolist() 
df2[df2.date>1]


,transaction_id,customer_id,date,result,ori_file,ori_sheet
0,1,2,2,2,2,2
1,2,2,2,2,2,2
2,3,2,2,2,2,2
3,4,2,2,2,2,2
4,5,2,2,2,2,2
...,...,...,...,...,...,...
3952,3953,2,2,2,2,2
3953,3954,2,2,2,2,2
3954,3955,2,2,2,2,2
3955,3956,2,2,2,2,2


In [23]:
df2.sort_values('Date', axis=0, ascending=False, inplace=True)
df2[0:10]

,Auth_ID,Date,Customer_ID
18,9000019,2,2
23,9000025,1,1
10,9000011,1,1
2,9000003,1,1
3,9000004,1,1
4,9000005,1,1
5,9000006,1,1
6,9000007,1,1
7,9000008,1,1
8,9000009,1,1


In [24]:
df2.to_excel("output/101_Output_Grouped.xlsx", sheet_name='Grouped')